In [1]:
# from utils_torsion_dataset_generator import *
from util_2nd_round_generator import *

RDKit WARNING: [00:08:40] Enabling RDKit 2019.09.3 jupyter extensions
Failed to import the evaluator package.


In [2]:
%%capture cap1 --no-stderr
# Create force field object
forcefield = ForceField('param_valence.offxml', allow_cosmetic_attributes=True)

# Create dictionaries storing molecules and attributes 
molecules_list_dict, molecule_attributes = read_aggregate_molecules("pfizer_optimization_inputs.json")

# List torsion parameters and effective rotations matched to each parameter from input molecule set
tid_molecules_list = gen_tid_molecules_list(molecule_attributes, molecules_list_dict, forcefield )

# Read pickle file containing data downloaded from qcarchive for reuse
gen2_torsiondrive_data = download_torsiondrive_data('OpenFF Gen 2 Torsion Set 3 Pfizer Discrepancy', output_pickle='pfizer_gen2_torsiondrive_data.pickle')
# List up pre-calculated torsions for re-use
gen2_tid_calculated_molecules_list, gen2_molecules_list_dict_from_td = gen_tid_calculated_molecules_list(gen2_torsiondrive_data, forcefield)   

# Read pickle file containing data downloaded from qcarchive for reuse
with open('pfizer_torsiondrive_data.pickle', 'rb') as pfile:
    torsiondrive_data = pickle.load(pfile)
tid_calculated_molecules_list, molecules_list_dict_from_td = gen_tid_calculated_molecules_list(torsiondrive_data, forcefield)   



Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 12, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 6, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 7, aromatic: False, chiral: True

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 16, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 7, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 10, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 11, aro

In [3]:
%%capture cap2 --no-stderr

# clustering each list of molecules and tid_molecules_list -> tid_clusters_list
# output: `tid_clusters_list[tid] = [..., {'cluster_label': N, 'torsions': [...]}, ...] `
tid_clusters_list = gen_tid_clusters_list_mod(tid_molecules_list, fptype=OEFPType_MACCS166)

In [4]:
# find if any cluster has pre-calculated torsion and add one more information 'reusable' in the dictionary 
# if 'reusable' == False, no reusable torsion detacted
# tid_clusters_list_detailed[tid] = [ {'cluster_label': N, 'torsions': [...], 'reusable': False or torsion_info}, ...]
tid_clusters_list_detailed = find_reusable_cluster(tid_clusters_list, gen2_tid_calculated_molecules_list)

In [5]:
# Convert linear dependency (data degenracy) into graph representaion
graph_reusable_set, graph_single_coverage_set, graph_multiple_coverage_sets = gen_graph_for_2nd_round(tid_clusters_list_detailed)

In [6]:
# randomized optimization procedure for minimization of data-degeneracy
selected , final_coverage, final_overlap, coverage_history, overlap_history = find_minimum_degeneracy_for_2nd_round(graph_reusable_set, graph_single_coverage_set, graph_multiple_coverage_sets)

In [7]:
%%capture cap3 --no-stderr

selected_rotations, molecules_list_dict_updated = select_rotations_for_2nd_round(tid_clusters_list_detailed, selected, molecules_list_dict,  tid_calculated_molecules_list=tid_calculated_molecules_list, molecules_list_dict_from_td=molecules_list_dict_from_td, first_round_tid_calculated_molecules_list=gen2_tid_calculated_molecules_list, first_round_molecules_list_dict_from_td=gen2_molecules_list_dict_from_td)

# Store selected molecules into json file
gen_json_for_2nd_round(selected_rotations, molecule_attributes, molecules_list_dict_updated, output_json='pfizer_2_selected_torsions.json')


In [8]:

with open('select.log', 'w') as f:
    f.write(cap1.stdout)
    f.write(cap2.stdout)
    f.write(cap3.stdout)